In [ ]:
# Export H3-aggregated PNGs grouped by date and by hour of day
from pathlib import Path
import re
import pandas as pd
import h3
from panogeo.mapplot import save_h3_basemap, merc_extent_from_center

# Config (kept consistent with previous cells)
OUTPUT_DIR = "./output"
PROVIDER = "japan_gsi_air"  # "carto", "osm", "esri-world", "japan_gsi_seamless", "japan_gsi_air", or XYZ URL
WIDTH_M = 150.0
HEIGHT_M = 150.0
H3_RES = 14
CAM_LAT = 35.16928165
CAM_LON = 136.90860244

# Optional fixed color scales per series (set *_VMAX=None to auto-compute)
DATE_VMIN, DATE_VMAX = 0, None
HOUR_VMIN, HOUR_VMAX = 0, None

# Fixed extent for visual consistency across all exports
fixed_extent = merc_extent_from_center(center_lat=CAM_LAT, center_lon=CAM_LON, width_m=WIDTH_M, height_m=HEIGHT_M)

# Load detections with lat/lon
geo_csv = str(Path(OUTPUT_DIR) / "all_people_geo_calibrated.csv")
df = pd.read_csv(geo_csv)
if df.empty:
    raise ValueError("No rows in geo CSV")
if not {"lat", "lon"}.issubset(df.columns):
    raise ValueError("CSV must contain 'lat' and 'lon' columns")

# Exclude detections within 2 m buffer of the camera position and report excluded count
BUFFER_M = 2.0
n_before = int(len(df))
if "range_m" in df.columns:
    _mask = df["range_m"].astype(float) > BUFFER_M
elif {"east_m", "north_m"}.issubset(df.columns):
    _rng = (df["east_m"].astype(float) ** 2 + df["north_m"].astype(float) ** 2) ** 0.5
    _mask = _rng > BUFFER_M
else:
    import numpy as np
    lat1 = np.radians(CAM_LAT)
    lon1 = np.radians(CAM_LON)
    lat2 = np.radians(df["lat"].astype(float).to_numpy())
    lon2 = np.radians(df["lon"].astype(float).to_numpy())
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    c = 2.0 * np.arctan2(np.sqrt(a), np.sqrt(1.0 - a))
    R = 6371000.0  # meters
    dist = R * c
    _mask = dist > BUFFER_M

excluded_count = int((~_mask).sum())
df = df[_mask].reset_index(drop=True)
print(f"Excluded by {BUFFER_M:.1f} m buffer: {excluded_count} of {n_before} total")

# Parse timestamp from image names like IMG_YYYYMMDD_HHMMSS_**_**.jpg → date, hour
_ts_re = re.compile(r"IMG_(\d{8})_(\d{6})_")

def _parse_date_hour(name: str):
    m = _ts_re.search(str(name))
    if not m:
        return None, None
    ymd, hms = m.groups()
    date = f"{ymd[:4]}-{ymd[4:6]}-{ymd[6:]}"
    hour = hms[:2]
    return date, hour

# Expand into two columns with apply (not map)
tmp = df["image"].apply(lambda x: pd.Series(_parse_date_hour(x), index=["date", "hour"]))
df[["date", "hour"]] = tmp
df = df.dropna(subset=["date", "hour"]).reset_index(drop=True)

# Output folders
out_date_dir = Path(OUTPUT_DIR) / "h3_by_date"
out_hour_dir = Path(OUTPUT_DIR) / "h3_by_hour"
out_date_dir.mkdir(parents=True, exist_ok=True)
out_hour_dir.mkdir(parents=True, exist_ok=True)

# Temporary CSV reused for each group
_tmp_csv = Path(OUTPUT_DIR) / "_tmp_h3_group.csv"

# Helper to save a group's H3 map
def _save_group(sub_df: pd.DataFrame, out_png: Path, vmin=None, vmax=None):
    if sub_df.empty:
        return None
    # Write only needed columns; keep extras if present
    sub_df.to_csv(_tmp_csv, index=False)
    return save_h3_basemap(
        geo_csv=str(_tmp_csv),
        out_png=str(out_png),
        provider=PROVIDER,
        zoom=18,
        h3_res=H3_RES,
        weight_col=None,  # or 'conf' to sum confidences
        alpha=0.5,
        dpi=150,
        margin_frac=0.10,
        fixed_extent_merc=fixed_extent,
        cmap="viridis",
        edgecolor="#00000000",  # transparent edges to avoid moiré/hatching
        linewidth=0.0,
        vmin=vmin,
        vmax=vmax,
        add_colorbar=True,
        colorbar_label="count" if ("conf" not in sub_df.columns) else "conf sum",
        rasterized=True,
    )

# Compute global vmax across all dates for consistent scale
unique_dates = sorted(df["date"].unique())
unique_hours = sorted(df["hour"].unique())

def _series_global_max(groups, keyname: str):
    if keyname == "date":
        keys = groups
    else:
        keys = groups
    global_max = 0.0
    for key in keys:
        sub = df[df[keyname] == key]
        if sub.empty:
            continue
        # Aggregate to H3 and get max value
        lat = sub["lat"].astype(float).to_numpy()
        lon = sub["lon"].astype(float).to_numpy()
        if hasattr(h3, "latlng_to_cell"):
            cells = [h3.latlng_to_cell(float(la), float(lo), int(H3_RES)) for la, lo in zip(lat, lon)]
        else:
            cells = [h3.geo_to_h3(float(la), float(lo), int(H3_RES)) for la, lo in zip(lat, lon)]
        tmp = pd.DataFrame({"h3": cells})
        agg = tmp.groupby("h3").size()
        if not agg.empty:
            m = float(agg.max())
            if m > global_max:
                global_max = m
    return global_max

_date_vmax = float(DATE_VMAX) if DATE_VMAX is not None else _series_global_max(unique_dates, "date")
_hour_vmax = float(HOUR_VMAX) if HOUR_VMAX is not None else _series_global_max(unique_hours, "hour")

# 1) Save per-date H3 maps
for date in unique_dates:
    sub = df[df["date"] == date]
    out_png = out_date_dir / f"people_summary_h3_r{H3_RES}_{date}_{PROVIDER}.png"
    try:
        out = _save_group(sub, out_png, vmin=DATE_VMIN, vmax=_date_vmax)
        if out:
            print(f"Saved: {out}")
    except Exception as e:
        print(f"Error saving date {date}: {e}")

# 2) Save per-hour-of-day H3 maps (across all dates)
for hour in unique_hours:
    sub = df[df["hour"] == hour]
    out_png = out_hour_dir / f"people_summary_h3_r{H3_RES}_hour-{hour}_{PROVIDER}.png"
    try:
        out = _save_group(sub, out_png, vmin=HOUR_VMIN, vmax=_hour_vmax)
        if out:
            print(f"Saved: {out}")
    except Exception as e:
        print(f"Error saving hour {hour}: {e}")

# Cleanup temp CSV (best-effort)
try:
    if _tmp_csv.exists():
        _tmp_csv.unlink()
except Exception:
    pass



In [1]:
# Prevent OpenMP runtime conflict in Windows (libiomp5md.dll vs libomp.dll)
import os
os.environ.setdefault("KMP_DUPLICATE_LIB_OK", "TRUE")
# Optional: reduce oversubscription
os.environ.setdefault("OMP_NUM_THREADS", "1")


'1'

# panogeo demo

End-to-end demo of detection and geolocation from panoramic imagery.

Requirements:
- Install environment: `conda env create -f environment.yml && conda activate panogeo`
- Install package: `pip install -e .`
- Prepare a folder of panoramas (equirectangular 2:1)



In [ ]:
# Interactive calibration UI demo
from panogeo import launch_calibration_ui

# Choose a pano image and a sensible map center (lat, lon)
ui = launch_calibration_ui(
    pano_path="data/images_shift/IMG_20250906_181558_00_263.jpg",
    map_center=(35.169237383755025, 136.90874779113642),  # TODO: set to your camera area
    map_zoom=19,
    display_width_px=1800,
    default_alt_m=0.0,
)
ui.display()

In [1]:
# Imports & paths
from pathlib import Path
from IPython.display import display

import pandas as pd

from panogeo.cli import main as panogeo_cli

# Set your paths
IMAGES_DIR = "./data/images"           # input panoramas
SHIFT_DIR  = "./data/images_shift"     # shifted output
OUTPUT_DIR = "./output"                # outputs
CALIB_CSV  = f"{OUTPUT_DIR}/calib_points.csv"  # provide or capture via your own tool

Path(SHIFT_DIR).mkdir(parents=True, exist_ok=True)
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

print("Setup complete.")


Setup complete.


In [3]:
# 1) Optional: yaw-shift panoramas so forward faces North (or desired yaw)
panogeo_cli(["shift-pano", "--in-dir", IMAGES_DIR, "--out-dir", SHIFT_DIR, "--degrees", "200"])


Saved 36 image(s) to ./data/images_shift


In [3]:
# 2) Detect people in panoramas with tiling
panogeo_cli([
    "detect",
    "--images-dir", SHIFT_DIR,
    "--output-dir", OUTPUT_DIR,
    "--model", "yolov8s.pt",
    "--conf", "0.50",
    "--iou", "0.50",
    "--containment-thr", "0.50",
    "--annotate",
    "--stamp",
    "--annotate-dir", OUTPUT_DIR + "/annotated",
    "--device", "cuda:0",
    "--batch-tiles", "16",
    "--fuse-model",
    "--half",
    "--bbox-color", "#FF00FF",
    # omit --half to keep outputs identical; add it later for extra speed
])

# Inspect aggregate detections
agg_csv = f"{OUTPUT_DIR}/detections/detections_all.csv"
df_det = pd.read_csv(agg_csv)
display(df_det.head())


YOLOv8s summary (fused): 72 layers, 11,156,544 parameters, 0 gradients, 28.6 GFLOPs
Detections saved: ./output\detections\detections_all.csv


,image,input_path,W,H,bbox_x1,bbox_y1,bbox_x2,bbox_y2,u_px,v_px,conf
0,IMG_20250906_110448_00_008,./data/images_shift\IMG_20250906_110448_00_008...,11904,5952,5119.000,3133.75,5231.0,3322.0,5175.0000,3322.0,0.886719
1,IMG_20250906_110448_00_008,./data/images_shift\IMG_20250906_110448_00_008...,11904,5952,2915.375,3134.50,3010.5,3282.5,2962.9375,3282.5,0.829102
2,IMG_20250906_110448_00_008,./data/images_shift\IMG_20250906_110448_00_008...,11904,5952,5253.000,3024.50,5292.0,3093.0,5272.5000,3093.0,0.819824
3,IMG_20250906_110448_00_008,./data/images_shift\IMG_20250906_110448_00_008...,11904,5952,3736.000,3098.00,3824.0,3308.0,3780.0000,3308.0,0.790527
4,IMG_20250906_110448_00_008,./data/images_shift\IMG_20250906_110448_00_008...,11904,5952,2482.000,4160.00,2819.0,4320.0,2650.5000,4320.0,0.787109


In [6]:
agg_csv = f"{OUTPUT_DIR}/detections/detections_all.csv"

In [4]:
# 3) Calibrate camera rotation from pixel↔geo pairs

from panogeo.cli import main as panogeo_cli

# Provide a CSV with columns: image,u_px,v_px,lon,lat[,alt_m][,W,H]
# Set your known camera location and altitude
CAM_LAT       = 35.16928165
CAM_LON       = 136.90860244
CAMERA_ALT_M  = 2.34       # camera altitude above local ground mean sea level (approx). Used only to set camera Z in ENU.
GROUND_ALT_M  = 0.0       # assumed ground altitude (MSL) when calib alt is absent

panogeo_cli([
    "calibrate",
    "--calib-csv", CALIB_CSV,
    "--cam-lat", str(CAM_LAT),
    "--cam-lon", str(CAM_LON),
    "--camera-alt-m", str(CAMERA_ALT_M),
    "--ground-alt-m", str(GROUND_ALT_M),
    "--output-dir", OUTPUT_DIR,
])


Saved calibration to: ./output\calibration_cam2enu.npz
yaw=-3.27°, pitch=-0.89°, roll=0.08°
CAM_LAT=35.16928165, CAM_LON=136.90860244, CAMERA_ALT_M=2.34


In [7]:
# 4) Geolocate detections to ENU and WGS84
calib_npz = f"{OUTPUT_DIR}/calibration_cam2enu.npz"
panogeo_cli([
    "geolocate",
    "--detections-csv", agg_csv,
    "--calibration", calib_npz,
    "--output-dir", OUTPUT_DIR,
])

geo_csv = f"{OUTPUT_DIR}/all_people_geo_calibrated.csv"
df_geo = pd.read_csv(geo_csv)
print(f"Rows: {len(df_geo)}")
display(df_geo.head())


Saved: ./output\all_people_xy_calibrated.csv
Saved: ./output\all_people_geo_calibrated.csv
Rows: 1068


,image,input_path,W,H,bbox_x1,bbox_y1,bbox_x2,bbox_y2,u_px,v_px,conf,east_m,north_m,up_m,lon,lat,range_m
0,IMG_20250906_110448_00_008,./data/images_shift\IMG_20250906_110448_00_008...,11904,5952,5118.838379,3133.548340,5230.365723,3321.864746,5174.602051,3321.864746,0.886842,-5.317374,10.497041,-2.34,136.908544,35.169376,11.997414
1,IMG_20250906_110448_00_008,./data/images_shift\IMG_20250906_110448_00_008...,11904,5952,2915.221191,3134.777588,3010.486084,3282.276855,2962.853638,3282.276855,0.828276,-14.453090,-0.963805,-2.34,136.908444,35.169273,14.672980
2,IMG_20250906_110448_00_008,./data/images_shift\IMG_20250906_110448_00_008...,11904,5952,5252.989258,3024.639160,5291.881836,3092.975098,5272.435547,3092.975098,0.819814,-12.449098,28.158383,-2.34,136.908466,35.169535,30.876369
3,IMG_20250906_110448_00_008,./data/images_shift\IMG_20250906_110448_00_008...,11904,5952,3735.639404,3098.066895,3824.963623,3307.867432,3780.301514,3307.867432,0.790180,-11.991505,4.578837,-2.34,136.908471,35.169323,13.047511
4,IMG_20250906_110448_00_008,./data/images_shift\IMG_20250906_110448_00_008...,11904,5952,2481.884766,4159.875488,2819.316162,4319.469727,2650.600464,4319.469727,0.786164,-2.670138,-0.658454,-2.34,136.908573,35.169276,3.610928


In [8]:
# 7) Folium all points (no aggregation)
try:
    import folium

    m_points_all = folium.Map(location=[float(df_geo["lat"].mean()), float(df_geo["lon"].mean())], zoom_start=19, tiles=None)
    # Google Satellite tiles
    folium.TileLayer(
        tiles="https://{s}.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
        attr="© Google",
        name="Google Satellite",
        subdomains=["mt0","mt1","mt2","mt3"],
        max_zoom=21,
    ).add_to(m_points_all)

    for r in df_geo.itertuples():
        conf = float(getattr(r, "conf", 1.0))
        folium.CircleMarker(
            location=[float(r.lat), float(r.lon)],
            radius=0.2,
            color="#2196f3",
            fill=True,
            fill_color="#2196f3",
            fill_opacity=0.6,
            popup=f"{r.image} (conf={conf:.2f})",
        ).add_to(m_points_all)
except Exception as e:
    print("Folium not available or error building all-points map:", e)


In [ ]:
display(m_points_all)


In [1]:
# 6b) Export PNG basemaps for all images with a fixed 100m x 100m basemap centered at camera
from pathlib import Path
from panogeo.mapplot import save_all_images_basemap, merc_extent_from_center

OUTPUT_DIR = "./output"                # outputs
CAM_LAT       = 35.16928165
CAM_LON       = 136.90860244

maps_dir = Path(OUTPUT_DIR) / "maps_all"
maps_dir.mkdir(parents=True, exist_ok=True)

PROVIDER = "japan_gsi_air"  # "carto", "osm", "esri-world" (Esri imagery), "japan_gsi", or XYZ URL

# Use CAM_LAT/CAM_LON set earlier in the notebook
WIDTH_M = 100.0
HEIGHT_M = 100.0

try:
    fixed_extent = merc_extent_from_center(center_lat=CAM_LAT, center_lon=CAM_LON, width_m=WIDTH_M, height_m=HEIGHT_M)
    saved = save_all_images_basemap(
        geo_csv=f"{OUTPUT_DIR}/all_people_geo_calibrated.csv",
        out_dir=str(maps_dir),
        provider=PROVIDER,
        zoom=18,  # or None to auto-select
        point_size=15.0,
        alpha=0.9,
        point_color="#FF00FF",
        dpi=150,
        margin_frac=0.10,
        fixed_extent_merc=fixed_extent,
        # stamp_timestamp=True,   # toggle here
    )
    print(f"Saved {len(saved)} images to {maps_dir}")
except Exception as e:
    print("PNG basemap export requires geopandas/contextily.")
    print("Install extras: pip install 'panogeo[geo]'")
    print("Error:", e)


Saved 36 images to output\maps_all


In [ ]:
# 6c) Export a single PNG summary map of all detections (fixed 100m x 100m)
from pathlib import Path
from panogeo.mapplot import save_points_basemap, merc_extent_from_center

PROVIDER = "carto"  # "carto", "osm", "esri-world" (Esri imagery), "japan_gsi", or XYZ URL
WIDTH_M = 100.0
HEIGHT_M = 100.0

summary_png = Path(OUTPUT_DIR) / f"people_summary_{PROVIDER}.png"

try:
    fixed_extent = merc_extent_from_center(center_lat=CAM_LAT, center_lon=CAM_LON, width_m=WIDTH_M, height_m=HEIGHT_M)
    out = save_points_basemap(
        geo_csv=f"{OUTPUT_DIR}/all_people_geo_calibrated.csv",
        out_png=str(summary_png),
        provider=PROVIDER,
        zoom=19,  # or None to auto-select
        point_size=16.0,
        alpha=0.9,
        dpi=150,
        margin_frac=0.10,
        image_name=None,
        fixed_extent_merc=fixed_extent,
    )
    print(f"Saved: {out}")
except Exception as e:
    print("PNG basemap export requires geopandas/contextily.")
    print("Install extras: pip install 'panogeo[geo]'")
    print("Error:", e)


Saved: output\people_summary_carto.png


In [ ]:
# 6d) Export an H3-aggregated PNG summary map of all detections
from pathlib import Path
from panogeo.mapplot import save_h3_basemap, merc_extent_from_center

OUTPUT_DIR = "./output"                # outputs
CAM_LAT       = 35.16928165
CAM_LON       = 136.90860244

PROVIDER = "carto"  # "carto", "osm", "esri-world" (Esri imagery), "japan_gsi", or XYZ URL
WIDTH_M = 150.0
HEIGHT_M = 150.0
H3_RES = 14   # smaller number = larger hexagons (e.g., 8..11)

h3_png = Path(OUTPUT_DIR) / f"people_summary_h3_r{H3_RES}_{PROVIDER}.png"

try:
    fixed_extent = merc_extent_from_center(center_lat=CAM_LAT, center_lon=CAM_LON, width_m=WIDTH_M, height_m=HEIGHT_M)
    out = save_h3_basemap(
        geo_csv=f"{OUTPUT_DIR}/all_people_geo_calibrated.csv",
        out_png=str(h3_png),
        provider=PROVIDER,
        zoom=19,  # or None to auto-select
        h3_res=H3_RES,
        weight_col=None,  # set to 'conf' to sum confidences instead of counts
        alpha=0.5,
        dpi=150,
        margin_frac=0.10,
        fixed_extent_merc=fixed_extent,
        cmap="viridis",
        edgecolor="#ffffff",
        linewidth=0.4,
    )
    print(f"Saved: {out}")
except Exception as e:
    print("H3 export requires 'h3' and geopandas/contextily.")
    print("Install: pip install h3 'panogeo[geo]'")
    print("Error:", e)


Saved: output\people_summary_h3_r14_carto.png


In [2]:
# 6d) Export an H3-aggregated PNG summary map of all detections
from pathlib import Path
from panogeo.mapplot import save_h3_basemap, merc_extent_from_center

OUTPUT_DIR = "./output"                # outputs
CAM_LAT       = 35.16928165
CAM_LON       = 136.90860244

PROVIDER = "japan_gsi_air"  # "carto", "osm", "esri-world", "japan_gsi_seamless", "japan_gsi_air", or XYZ URL
WIDTH_M = 150.0
HEIGHT_M = 150.0
H3_RES = 14   # smaller number = larger hexagons (e.g., 8..11)

h3_png = Path(OUTPUT_DIR) / f"people_summary_h3_r{H3_RES}_{PROVIDER}.png"

try:
    fixed_extent = merc_extent_from_center(center_lat=CAM_LAT, center_lon=CAM_LON, width_m=WIDTH_M, height_m=HEIGHT_M)
    out = save_h3_basemap(
        geo_csv=f"{OUTPUT_DIR}/all_people_geo_calibrated.csv",
        out_png=str(h3_png),
        provider=PROVIDER,
        zoom=18,  # or None to auto-select
        h3_res=H3_RES,
        weight_col=None,  # set to 'conf' to sum confidences instead of counts
        alpha=0.5,
        dpi=150,
        margin_frac=0.10,
        fixed_extent_merc=fixed_extent,
        cmap="viridis",
        edgecolor="#ffffff",
        linewidth=0.4,
    )
    print(f"Saved: {out}")
except Exception as e:
    # print("H3 export requires 'h3' and geopandas/contextily.")
    # print("Install: pip install h3 'panogeo[geo]'")
    print("Error:", e)


Saved: output\people_summary_h3_r14_japan_gsi_air.png


In [ ]:
from pathlib import Path
from PIL import Image, ImageOps

# Combine annotated images with corresponding map inset (bigger, white edge, slight offset)
annotated_dir = Path('output/annotated')
map_dirs = [Path('output/maps_all'), Path('output/maps')]
output_dir = Path('output/combined')
output_dir.mkdir(parents=True, exist_ok=True)

map_priority = ['japan_gsi_air', 'carto', 'osm']

created = 0
skipped = []

for ann_path in sorted(annotated_dir.glob('*_annotated.jpg')):
    base = ann_path.name[:-len('_annotated.jpg')]

    # Choose best map
    map_path = None
    for layer in map_priority:
        candidate = None
        for d in map_dirs:
            p = d / f'{base}_{layer}.png'
            if p.exists():
                candidate = p
                break
        if candidate:
            map_path = candidate
            break
    if map_path is None:
        for d in map_dirs:
            matches = sorted(d.glob(f'{base}_*.png'))
            if matches:
                map_path = matches[0]
                break
    if map_path is None:
        skipped.append((ann_path.name, 'no map found'))
        continue

    try:
        annotated_img = Image.open(ann_path).convert('RGB')
        map_img = Image.open(map_path).convert('RGB')

        # Height of map inset = panorama height / 2.2
        target_h = max(1, int(annotated_img.height / 2.2))
        target_w = max(1, int(map_img.width * (target_h / map_img.height)))
        inset = map_img.resize((target_w, target_h), Image.LANCZOS)

        # White edge border
        inset = ImageOps.expand(inset, border=8, fill='white')

        # Slight offset from top-left
        offset_x = max(10, int(annotated_img.width * 0.015))
        offset_y = max(10, int(annotated_img.height * 0.015))

        combined = annotated_img.copy()
        combined.paste(inset, (offset_x, offset_y))

        out_path = output_dir / f'{base}_annotated_with_map.jpg'
        combined.save(out_path, quality=92)
        created += 1
    except Exception as e:
        skipped.append((ann_path.name, str(e)))

print(f'Created: {created}')
if skipped:
    print('Skipped:')
    for name, reason in skipped[:10]:
        print(' -', name, '->', reason)
    if len(skipped) > 10:
        print(f' ... and {len(skipped) - 10} more')

# Preview one
try:
    from IPython.display import display
    examples = sorted(output_dir.glob('*_annotated_with_map.jpg'))
    if examples:
        display(Image.open(examples[0]))
except Exception:
    pass

